In [23]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from lib.m import *
from lib.eda_visualization import *

In [24]:
df = pd.read_csv("df_complete.csv").iloc[:,3:]
df

c:\Users\matan\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (18,88) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,company_name,company_about,founded,business model,employees,funding stage,raised,product stage,status,geographical markets,...,tag_e-mobility,targetmarket_public-transportation,targetmarket_smart-mobility,tag_hydraulic-drive,tag_hud,tag_simulation-software,tag_luggage,tag_traffic-violations,tag_car-audio,tag_trip
0,Tastewise,Tastewise is an AI platform designed to help f...,7/2017,B2B,51-200,ROUND A,$21.5M,Released,active,"australia, canada, france, india, united kingd...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wilk Technologies,Wilk is dedicated to revolutionizing the dairy...,6/2018,"B2B, B2B2C",11-50,Public,$4.69M,R&D,active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eco Pack Green Box,Eco Pack Green Box has developed and patented ...,3/2008,B2B,11-50,Revenue Financed,NaN,Released,not_active,"canada, mexico, spain, united states",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BeeHero,BeeHero has developed a platform that can pred...,10/2017,B2B,1-10,ROUND A,$24M,Released,active,"global, united states",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cham Foods,Cham Foods is a multinational company with man...,12/1970,"B2B, B2B2C",11-50,Public,NaN,Released,active,"north america, europe, global, france, germany...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13043,TriEye,TriEye is a fabless semiconductor company that...,11/2017,B2B,51-200,ROUND A,$96M,R&D,active,global,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13044,LYNX Smartcars,LYNX is developing software for connected and ...,1/2016,B2B,1-10,Bootstrapped,NaN,R&D,not_active,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13045,Deeyook Location Technologies,Deeyook seeks to redefine location technology ...,3/2017,B2B,11-50,Seed,NaN,Released,active,global,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13046,SafeCue,SafeCue combines the power of deep learning wi...,1/2016,B2B,1-10,Seed,$500K,Beta,not_active,"asia, germany, india, united states",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_test = df.copy()
df_test.shape

(13048, 2868)

In [26]:
tag_cols = [col for col in df_test.columns if col.startswith('tag_')]
targetmarket_cols = [col for col in df_test.columns if col.startswith('targetmarket_')]
sector_list = [col for col in df_test.columns if col.startswith("sector_")]
target_ind_list = [col  for col in df_test.columns if col.startswith("target_industry_")]
technology_list = [col  for col in df_test.columns if col.startswith("core_technology_")]


In [27]:
df_test[tag_cols] = df_test[tag_cols].fillna(0)
df_test[targetmarket_cols] = df_test[targetmarket_cols].fillna(0)
df_test[sector_list] = df_test[sector_list].fillna(0)
df_test[target_ind_list] = df_test[target_ind_list].fillna(0)
df_test[technology_list] = df_test[technology_list].fillna(0)


In [28]:
# df_test.tag_vec[0][0]

In [39]:
def vectorize_and_replace(df, cols: list, prefix: str):
    """Vectorize the {cols} columns in {dataframe}\n
    and returns a {dataframe} copy with the vector after removing the {cols}\n
    new col name is {prefix}_vec """
    # df = dataframe.copy()

    lb = preprocessing.LabelBinarizer()
    vec = lb.fit_transform(df[cols]).tolist()

    # vec = [int(x) for x in vec.split(',')]


    df[f"{prefix}_vec"] =  vec #np.array([vec], dtype = int)
    df = df.drop(cols, axis = 1)
    return df

vectorize

In [40]:
df_test = vectorize_and_replace(df_test, tag_cols, "tag")
df_test = vectorize_and_replace(df_test, targetmarket_cols, "targetmarket")
df_test = vectorize_and_replace(df_test, sector_list, "sector")
df_test = vectorize_and_replace(df_test, target_ind_list, "target_ind")
df_test = vectorize_and_replace(df_test, technology_list, "tech")
df_test.shape

(13048, 23)

Replacing null values to 0

In [41]:
df_test['raised'] = df_test['raised'].fillna(0)
df_test['total_rounds'] = df_test['total_rounds'].fillna(0)
df_test['investors'] = df_test['investors'].fillna(0)
df_test['ipo_price'] = df_test['ipo_price'].fillna(0)


In [42]:
df_test.status.replace({'active' : 1, 'not_active' : 0 }, inplace=True)

Removing unused columns

In [43]:
df_test.drop(['use cases','academic spin-off','total_raised'], axis = 1, inplace = True)
df_test.shape

(13048, 20)

Decoding str to numeric value

In [44]:
df_test['ipo_price'] = conv_to_float(df_test["ipo_price"])
df_test['raised'] = conv_to_float(df_test["raised"])

c:\Users\matan\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Converting the numeric colums

In [45]:

df_test['business model']= preprocessing.LabelEncoder().fit_transform(df_test['business model'])

In [46]:
df_test['raised'] = df_test['raised'].astype('float')
df_test['total_rounds'] = df_test['total_rounds'].astype('int')
df_test['investors'] = df_test['investors'].astype('int')
df_test['ipo_price'] = df_test['ipo_price'].astype('float')
df_test['business model'] = df_test['business model'].astype('int')

In [47]:
df_test.ipo_price.mean()

841852.5444512569

In [48]:
df_test.loc[(df_test["status"]==1)&(df_test['raised']>4000000), 'suceeded'] = 1
df_test.loc[(df_test["status"]==0)|(df_test['raised']<=4000000), 'suceeded'] = 0

In [49]:
df_test.to_csv('cleaned_complete.csv')

In [50]:
# df_test.loc["succeeded"] = df_test[(df_test['status'] == 1) & (df_test['raised'] > 5000000)]
df_test.shape

(13048, 21)

In [61]:

xtrain, xtest, ytrain, ytest = train_test_split(df_test.iloc[:,:-1].select_dtypes(include=np.number), df_test.iloc[:,-1])


In [52]:
df_test.iloc[2930,:]

company_name                                                    SolarEdge
company_about           SolarEdge Technologies invented an inverter so...
founded                                                            8/2006
business model                                                          2
employees                                                            500+
funding stage                                                      Public
raised                                                        124700000.0
product stage                                                    Released
status                                                                  1
geographical markets    europe, asia, africa, global, australia, cambo...
products                                                              NaN
fund_stage                                                         Public
total_rounds                                                            6
investors                             

In [72]:
# TODO maybe a function for each vec col?
#      or maybe not vectorize !?

lst_vec = df_test.iloc[:,-6].tolist()
df_vec = pd.DataFrame(lst_vec)
df_vec.shape
xtrain, xtest, ytrain, ytest = train_test_split(df_vec, df_test.iloc[:,-1])

In [73]:
df_vec.duplicated().sum()

3049

In [74]:
lr = LogisticRegression()
lr.fit(xtrain,ytrain)

LogisticRegression()

In [75]:
ytrain_pred = lr.predict(xtrain)
ytest_pred = lr.predict(xtest)

In [64]:
metrics.confusion_matrix(ytrain_pred, ytrain)

array([[6234,    0],
       [2121, 1431]], dtype=int64)

In [76]:
print("Train results:")
print("accuracy is:",metrics.accuracy_score(ytrain_pred, ytrain))
print("precision is:",metrics.precision_score(ytrain_pred, ytrain))
print("recall is:",metrics.recall_score(ytrain_pred, ytrain))
print("f1 is:",metrics.f1_score(ytrain_pred, ytrain))
print("---------------------")
print("Test results:")
print("accuracy is:",metrics.accuracy_score(ytest_pred, ytest))
print("precision is:",metrics.precision_score(ytest_pred, ytest))
print("recall is:",metrics.recall_score(ytest_pred, ytest))
print("f1 is:",metrics.f1_score(ytest_pred, ytest))

Train results:
accuracy is: 0.9119149805845085
precision is: 0.4602836879432624
recall is: 0.8653333333333333
f1 is: 0.6009259259259259
---------------------
Test results:
accuracy is: 0.865113427345187
precision is: 0.3045267489711934
recall is: 0.592
f1 is: 0.4021739130434782


<strong>This is the results with vectorize</strong>